In [ ]:
import kagglehub
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import nltk
import string
import os
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

c:\Users\misss\Formation\Projets\AnalyseSentiments\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\misss\.cache\kagglehub\datasets\kazanova\sentiment140\versions\2


In [ ]:


file_path = r"C:\Users\misss\.cache\kagglehub\datasets\kazanova\sentiment140\versions\2\training.1600000.processed.noemoticon.csv"

columns = ['target', 'id', 'date', 'flag', 'user', 'text']
data = pd.read_csv(file_path, encoding='latin-1', names=columns)

data = data[['target', 'text']]

# Convertir les sentiments en binaire (0 = négatif, 1 = positif)
data['target'] = data['target'].replace({4: 1})


print(data.head())



   target                                               text
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  is upset that he can't update his Facebook by ...
2       0  @Kenichan I dived many times for the ball. Man...
3       0    my whole body feels itchy and like its on fire 
4       0  @nationwideclass no, it's not behaving at all....


In [4]:
data.shape

(1600000, 2)

In [ ]:

nltk.download('stopwords', download_dir='C:/Users/misss/Formation/Projets/AnalyseSentiments/nltk_data')
nltk.download('punkt_tab', download_dir='C:/Users/misss/Formation/Projets/AnalyseSentiments/nltk_data')
nltk.download('wordnet', download_dir='C:/Users/misss/Formation/Projets/AnalyseSentiments/nltk_data')
nltk.download('word_tokenize', download_dir='C:/Users/misss/Formation/Projets/AnalyseSentiments/nltk_data')
nltk.data.path.append('C:/Users/misss/Formation/Projets/AnalyseSentiments/nltk_data')


[nltk_data] Downloading package stopwords to C:/Users/misss/Formation/
[nltk_data]     Projets/AnalyseSentiments/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt_tab to C:/Users/misss/Formation/
[nltk_data]     Projets/AnalyseSentiments/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to C:/Users/misss/Formation/Pr
[nltk_data]     ojets/AnalyseSentiments/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Error loading word_tokenize: Package 'word_tokenize' not
[nltk_data]     found in index


In [ ]:


lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
OOV_TOKEN = "<OOV>"

# Fonction de nettoyage général (suppression des mentions, URL et conversion en minuscule)
def clean_text(text):
    text = re.sub(r"@\w+", "", text)  # Supprimer les mentions
    text = re.sub(r"http\S+", "", text)  # Supprimer les URLs
    text = text.lower()  # Convertir en minuscule
    return text

#  Nettoyage pour TF-IDF (lemmatisation et suppression des stopwords) avec gestion OOV
def preprocess_tfidf(text):
    if isinstance(text, str):
        text = re.sub(r'[^\w\s]', '', text)  # Supprimer la ponctuation
        tokens = word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) if word not in stop_words else OOV_TOKEN for word in tokens]  
        return " ".join(tokens)  # Retourner sous forme de phrase entière
    return ""

#  Nettoyage pour Word2Vec / FastText (avec stopwords et OOV)
def preprocess_lem_with_stopwords(text):
    if isinstance(text, str):
        text = re.sub(r'[^\w\s]', '', text)
        tokens = word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) if word not in stop_words else OOV_TOKEN for word in tokens]
        return tokens  # Garder la liste de tokens
    return []

#  Nettoyage pour Word2Vec / FastText (sans stopwords et avec OOV)
def preprocess_lem_no_stopwords(text):
    if isinstance(text, str):
        text = re.sub(r"[^\w\s]", "", text)
        tokens = word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens]  
        return tokens
    return []

#  Nettoyage pour Word Embeddings avec stemming et stopwords
def preprocess_stem_with_stopwords(text):
    if isinstance(text, str):
        text = re.sub(r"[^\w\s]", "", text)
        tokens = word_tokenize(text)
        tokens = [stemmer.stem(word) if word not in stop_words else OOV_TOKEN for word in tokens]
        return tokens
    return []

#  Nettoyage pour Word Embeddings avec stemming sans stopwords
def preprocess_stem_no_stopwords(text):
    if isinstance(text, str):
        text = re.sub(r"[^\w\s]", "", text)
        tokens = word_tokenize(text)
        tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]  
        return tokens
    return []

# Nettoyage pour BERT (on garde le texte brut mais on ajoute un OOV token explicite)
def preprocess_bert(text):
    if isinstance(text, str):
        text = re.sub(r"@\w+", OOV_TOKEN, text)  # Remplace les mentions
        text = re.sub(r"http\S+", OOV_TOKEN, text)  # Remplace les URLs
        text = text.lower()
    return text

# Appliquer les nettoyages et créer les nouvelles colonnes
data["text_cleaned"] = data["text"].apply(clean_text)
data["text_tfidf"] = data["text_cleaned"].apply(preprocess_tfidf)
data["text_lem_with_stopwords"] = data["text_cleaned"].apply(preprocess_lem_with_stopwords)
data["text_lem_no_stopwords"] = data["text_cleaned"].apply(preprocess_lem_no_stopwords)
data["text_stem_with_stopwords"] = data["text_cleaned"].apply(preprocess_stem_with_stopwords)
data["text_stem_no_stopwords"] = data["text_cleaned"].apply(preprocess_stem_no_stopwords)
data["text_bert"] = data["text"].apply(preprocess_bert)

# Affichage des premières lignes
print(data.head())


   target                                               text  \
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...   
1       0  is upset that he can't update his Facebook by ...   
2       0  @Kenichan I dived many times for the ball. Man...   
3       0    my whole body feels itchy and like its on fire    
4       0  @nationwideclass no, it's not behaving at all....   

                                        text_cleaned  \
0    - awww, that's a bummer.  you shoulda got da...   
1  is upset that he can't update his facebook by ...   
2   i dived many times for the ball. managed to s...   
3    my whole body feels itchy and like its on fire    
4   no, it's not behaving at all. i'm mad. why am...   

                                          text_tfidf  \
0  awww thats bummer shoulda got david carr third...   
1  upset cant update facebook texting might cry r...   
2  dived many time ball managed save 50 rest go b...   
3                    whole body feel itchy like fire  

In [8]:

# Sauve les données nettoyées dans un CSV
data.to_csv('tweets_cleaned.csv', index=False)